In [ ]:
import json
import nltk
import pandas as pd
import operator
import gensim
from gensim import corpora
from gensim.models import LdaModel
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download('wordnet')
from nltk.corpus import stopwords 
import gzip
import bz2
import csv
import glob
from datetime import datetime
import os
import zipfile
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
os.getcwd()
#os.chdir('/')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


'/content'

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving preprocessing_project_data_w.csv to preprocessing_project_data_w.csv
User uploaded file "preprocessing_project_data_w.csv" with length 13044142 bytes


In [ ]:
# URL 
# https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/  explanation of BOW

In [ ]:
def BOW_preprocessing(df1,df_col):
    df = df1[df_col.notnull()]
    tokenizer = RegexpTokenizer(r'\w+')
    doc_1 = df_col[19000]
    #print('doc_1:' ,doc_1)
    tokens = tokenizer.tokenize(doc_1.lower())
    #print('tokens:',tokens)
    nltk_stpwd = stopwords.words('english')
    customise_stop_Words = ['is','the',':','|','Model','.',']']
    nltk_stpwd.extend(customise_stop_Words)
    nltk_stpwd.remove("other")
    stopped_tokens = [token for token in tokens if not token in nltk_stpwd]
    lemmatizer = WordNetLemmatizer()
    Lemmatizer_tokens = [lemmatizer.lemmatize(token) for token in stopped_tokens]
    lem_sponser = df.shape[0]
    doc_set = [df_col[i] for i in range(len(df))]
    texts_sponser= []
    for doc in doc_set:
        tokens = tokenizer.tokenize(doc.lower())
        stopped_tokens = [token for token in tokens if not token in nltk_stpwd]
        Lemmatizer_tokens = [lemmatizer.lemmatize(token) for token in stopped_tokens]
        texts_sponser.append(Lemmatizer_tokens)
    texts_dict = corpora.Dictionary(texts_sponser)
    texts_dict.save('BOW.dict_2')
    texts_dict.filter_extremes(no_below=20, no_above=0.10)
    # Converting the dictionary to bag of words
    corpus = [texts_dict.doc2bow(text) for text in texts_sponser]
    model = Tokenizer()
    model.fit_on_texts(texts_sponser)
    rep = model.texts_to_matrix(texts_sponser, mode='count')
    return rep, corpus

In [ ]:
df1=pd.read_csv('preprocessing_project_data_w.csv')


In [ ]:
BOW_SD,corpust=BOW_preprocessing(df1,df1.StudyDesign)
BOW_FB,corpus=BOW_preprocessing(df1,df1.FundedBys)


In [ ]:
# Study_Design
dft_SD=(BOW_SD).tolist()
df_SD = pd.DataFrame(dft_SD)
df_SD.columns = [ 'StudyDesign' + str(col) for col in df_SD.columns]
df_SD=df_SD.drop(['StudyDesign0'], axis=1)


In [ ]:
dft_FB=(BOW_FB).tolist()
df_FB = pd.DataFrame(dft_FB)
df_FB.columns = [ 'FundedBys' + str(col) for col in df_FB.columns]
df_FB=df_FB.drop(['FundedBys0'], axis=1)

In [ ]:
#df_col = pd.concat([df1['Target'],df1['Enrollment'],df1['Gender'],df1['StudyType'],dftt,df_SD], axis=1)
#df_col.to_csv('pre_process_Encoded_data.csv',index=False)

In [ ]:
df_FB=df_FB.rename(columns={"FundedBys1": "FundedBys_Others","FundedBys2": "FundedBys_Industries",
                           "FundedBys3": "FundedBys_NIH","FundedBys4":"FundedBys_USA_FED"})

In [ ]:
df_SD

,StudyDesign1,StudyDesign2,StudyDesign3,StudyDesign4,StudyDesign5,StudyDesign6,StudyDesign7,StudyDesign8,StudyDesign9,StudyDesign10,StudyDesign11,StudyDesign12,StudyDesign13,StudyDesign14,StudyDesign15,StudyDesign16,StudyDesign17,StudyDesign18,StudyDesign19,StudyDesign20,StudyDesign21,StudyDesign22,StudyDesign23,StudyDesign24,StudyDesign25,StudyDesign26,StudyDesign27,StudyDesign28,StudyDesign29,StudyDesign30,StudyDesign31,StudyDesign32,StudyDesign33,StudyDesign34,StudyDesign35,StudyDesign36,StudyDesign37,StudyDesign38,StudyDesign39,StudyDesign40,StudyDesign41,StudyDesign42,StudyDesign43,StudyDesign44,StudyDesign45,StudyDesign46,StudyDesign47,StudyDesign48,StudyDesign49,StudyDesign50,StudyDesign51,StudyDesign52,StudyDesign53,StudyDesign54,StudyDesign55,StudyDesign56
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19996,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19997,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19998,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# https://stackoverflow.com/questions/47193230/python-pandas-how-to-assign-keywords-extracted-from-a-column-to-another-column
df_PP = df1['StudyDesign'].str.extract('(Basic Science|Device Feasibility|Diagnostic|Educational/Counseling/Training|Health Services Research|Other|Prevention|Screening|Supportive Care|	Treatment)', expand = False)

In [ ]:
df_col = pd.concat([df1['Target'],df1['Enrollment'],df1['Gender'],df1['StudyType'],df_FB,df_SD,df_PP], axis=1)

In [ ]:
df_col = df_col.rename(columns={'StudyDesign': 'PrimaryPurpose'})
df_col['PrimaryPurpose']=df_col['PrimaryPurpose'].fillna('missing')

In [ ]:
df_col = pd.get_dummies(df_col,prefix=['PrimaryPurpose'])
df_col = df_col.rename(columns={'PrimaryPurpose_Basic Science': 'BasicScience',
                                'PrimaryPurpose_Device Feasibility' : 'DeviceFeasibility',
                                'PrimaryPurpose_Diagnostic' : 'Diagnostic',
                                'PrimaryPurpose_Health Services Research' : 'Health Services Research',
                                'PrimaryPurpose_Other' : 'Other',
                                'PrimaryPurpose_Prevention' : 'Prevention',
                                'PrimaryPurpose_Screening' : 'Screening',
                                'PrimaryPurpose_Supportive Care' : 'SupportiveCare',
                                'PrimaryPurpose_missing' : 'PrimaryPurposeMissing'})

In [ ]:
df_col

,Target,Enrollment,Gender,StudyType,FundedBys_Others,FundedBys_Industries,FundedBys_NIH,FundedBys_USA_FED,StudyDesign1,StudyDesign2,StudyDesign3,StudyDesign4,StudyDesign5,StudyDesign6,StudyDesign7,StudyDesign8,StudyDesign9,StudyDesign10,StudyDesign11,StudyDesign12,StudyDesign13,StudyDesign14,StudyDesign15,StudyDesign16,StudyDesign17,StudyDesign18,StudyDesign19,StudyDesign20,StudyDesign21,StudyDesign22,StudyDesign23,StudyDesign24,StudyDesign25,StudyDesign26,StudyDesign27,StudyDesign28,StudyDesign29,StudyDesign30,StudyDesign31,StudyDesign32,StudyDesign33,StudyDesign34,StudyDesign35,StudyDesign36,StudyDesign37,StudyDesign38,StudyDesign39,StudyDesign40,StudyDesign41,StudyDesign42,StudyDesign43,StudyDesign44,StudyDesign45,StudyDesign46,StudyDesign47,StudyDesign48,StudyDesign49,StudyDesign50,StudyDesign51,StudyDesign52,StudyDesign53,StudyDesign54,StudyDesign55,StudyDesign56,BasicScience,DeviceFeasibility,Diagnostic,Health Services Research,Other,Prevention,Screening,SupportiveCare,PrimaryPurposeMissing
0,0,54.0,0,0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,1
1,0,358.0,0,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,1
2,0,30.0,1,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,1
3,0,69.0,0,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,1
4,0,124.0,1,0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,67.0,0,0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,1
19996,1,14.0,0,0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0
19997,1,178.0,0,0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,1
19998,1,52.0,0,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,1


In [ ]:
df_col.to_csv('prepreocess_bow_encoded_data_v2.csv',index=False)
#df_col.to_csv('prepreocess_bow_encoded_data.csv',index=False)

In [ ]:
df_col.columns

Index(['Target', 'Enrollment', 'Gender', 'StudyType', 'FundedBys_Others',
       'FundedBys_Industries', 'FundedBys_NIH', 'FundedBys_USA_FED',
       'StudyDesign1', 'StudyDesign2', 'StudyDesign3', 'StudyDesign4',
       'StudyDesign5', 'StudyDesign6', 'StudyDesign7', 'StudyDesign8',
       'StudyDesign9', 'StudyDesign10', 'StudyDesign11', 'StudyDesign12',
       'StudyDesign13', 'StudyDesign14', 'StudyDesign15', 'StudyDesign16',
       'StudyDesign17', 'StudyDesign18', 'StudyDesign19', 'StudyDesign20',
       'StudyDesign21', 'StudyDesign22', 'StudyDesign23', 'StudyDesign24',
       'StudyDesign25', 'StudyDesign26', 'StudyDesign27', 'StudyDesign28',
       'StudyDesign29', 'StudyDesign30', 'StudyDesign31', 'StudyDesign32',
       'StudyDesign33', 'StudyDesign34', 'StudyDesign35', 'StudyDesign36',
       'StudyDesign37', 'StudyDesign38', 'StudyDesign39', 'StudyDesign40',
       'StudyDesign41', 'StudyDesign42', 'StudyDesign43', 'StudyDesign44',
       'StudyDesign45', 'StudyDesign46', 